In [170]:
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.neighbors import KernelDensity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
df = pd.read_csv("A0.75_C0.40_S0.40_W0.50.csv")

In [4]:
print(df)

       frame  id    x    y  image_index  angle
0          0   0  638  575            0   -138
1          0   1  371  340            0   -182
2          0   2  590   68            0     35
3          0   3  630  325            0   -100
4          0   4  541  169            0     -4
...      ...  ..  ...  ...          ...    ...
30045    600  45  319  742            1   -182
30046    600  46  159  559            1    -90
30047    600  47  535  561            1   -232
30048    600  48   69  518            1    -74
30049    600  49  555  566            1   -234

[30050 rows x 6 columns]


In [ ]:
in_flocks = []
angles = []
clusters = []

for frame in tqdm(range(max(df["frame"]))):
    in_flocks.append(df["image_index"][(df["frame"]==frame) & (df["image_index"]==1)].count())
    
    angle = np.array(list(df["angle"][df["frame"]==frame])).reshape(-1, 1)
    angles.append(angle)
    
    scores = []
    ds = df[["x","y"]][df["frame"]==frame].to_numpy()
    for k in range(2, 50):
        model = KMeans(n_clusters=k)
        model.fit(ds)
        pred = model.predict(ds)
        scores.append(silhouette_score(ds, pred))
    clusters.append(np.argmax(scores)+2)
    
    

 40%|███████████████████████████████▊                                                | 239/600 [12:39<18:47,  3.12s/it]

In [ ]:
fig, ax = plt.subplots()

ax.vlines(162,-1,2,colors = "r")
ax.vlines(337,-1,2,colors = "r")
ax.vlines(446,-1,2,colors = "r")

ax.plot(range(max(df["frame"])),[x/50 for x in in_flocks])

ax.set_title("Percentage of Birds in a Flock over Time")
ax.set_ylabel("Percentahe of birds in a flock")
ax.set_xlabel("Frame")

plt.ylim([0,1])

In [ ]:
def moving_average(x, n=10) :
    ret = np.cumsum(x, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
fig, ax = plt.subplots()
ax.vlines(162,-2,22,colors = "r")
ax.vlines(337,-1,22,colors = "r")
ax.vlines(446,-1,22,colors = "r")


ax.plot(range(max(df["frame"])),moving_average(clusters,n = 60) ,color = "g")
ax.plot(range(max(df["frame"])),moving_average(clusters), color = "b")
ax.plot(range(max(df["frame"])),clusters)
ax.set_title("Clusters per Frame")
ax.set_ylabel("Amount of Clusters")
ax.set_xlabel("Frame")


plt.ylim([2,20])